<a href="https://colab.research.google.com/github/MichelleElizabethK/Abstractive-User-Review-Consolidation/blob/master/Module2.5.1ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%time
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
pd.set_option('display.max_colwidth',1000)

df = pd.read_csv(F"/content/gdrive/My Drive/PROJECT | S7-S8/Data/training.csv", engine='python', names=["target", "ids", "date", "flag", "user", "text"])
df = df[:300000].append(df[-300000:])
print(df)
print(df.columns)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
         target  ...                                                                                                                 text
0             0  ...  @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
1             0  ...      is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!
2             0  ...                            @

In [0]:
%%time
from bs4 import BeautifulSoup
import urllib3
import json
http=urllib3.PoolManager()
Abbr_dict={}

#Function to get the Slangs from https://www.noslang.com/dictionary/
def getAbbr(alpha):
    global Abbr_dict
    r=http.request('GET','https://www.noslang.com/dictionary/'+alpha)
    soup=BeautifulSoup(r.data,'html.parser')
    
    for i in soup.findAll('div',{'class':'dictionary-word'}): 

        abbr=i.find('abbr')['title']
        Abbr_dict[i.find('span').text[:-2]]=abbr

#Generating a-z
linkDict=[]
for x in range(97,123):
    linkDict.append(chr(x))

#Creating Links for https://www.noslang.com/dictionary/a...https://www.noslang.com/dictionary/b....etc
for i in linkDict:
    getAbbr(i)

print(Abbr_dict)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

{'a$$': 'ass', 'a&f': 'always and forever', "a'ight": 'alright', 'a.i.m.': 'aol instant messanger', 'a/l': 'age and location', 'a/m': 'away message', 'a/s/l': 'age,sex,location', 'a/s/l/p': 'age/sex/location/picture', 'a/s/l/r': 'age, sex, location, race', 'a1t': 'anyone there', 'A2A': 'Ask to Answer', 'a3': 'anyplace, anywhere, anytime', 'a4u': 'all for you', 'A7X': 'Avenged Sevenfold', 'aaaaa': 'American Assosciation Against Acronym Abuse', 'aabf': 'as a best friend', 'aaf': 'as a friend', 'aak': 'Alive and Kicking', 'aamof': 'as a matter of fact', 'aatf': 'always and totally forever', 'aatw': 'all around the world', 'abd': 'Already Been Done', 'abend': 'absent by enforced net deprivation', 'abft': 'About f**king Time', 'aboot': 'about', 'abreev': 'abbreviation', 'absnt': 'absent', 'abt': 'about', 'abwt': 'about', 'acc': 'account', 'acct': 'account', 'acgaf': "Absolutely couldn't give a f**k", 'ack': 'acknowledged', 'addy': 'address', 'adhd': 'Attention Deficit Hyperactivity Disorder

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [0]:
%%time
from bs4 import BeautifulSoup
import re
import nltk
from spacy.lang.en.stop_words import STOP_WORDS
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download("punkt")
nltk.download("wordnet")
stop_words = list(STOP_WORDS)
wordnet_lemmatizer = WordNetLemmatizer()

def text_cleaner(strg):
    newString = strg.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([Abbr_dict[t] if t in Abbr_dict else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    newString = re.sub('[m]{2,}', 'mm', newString)
    newString = re.sub('http|https', "", newString)
    tokens = []
    for w in newString.split():
      if (w not in stop_words) and (w not in "?:!.,;"):
        if w in Abbr_dict.keys() and len(Abbr_dict[w])<2 :
          tokens += [wordnet_lemmatizer.lemmatize(Abbr_dict[w])]
        elif w in Abbr_dict.keys():
          tokens += text_cleaner(Abbr_dict[w]).split()
        else :
          tokens += [wordnet_lemmatizer.lemmatize(w)]
    return (" ".join(tokens)).strip()

df['PreprocessedText'] = df.text.apply(lambda x: text_cleaner(x))
print(df)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
         target  ...                                                    PreprocessedText
0             0  ...  switchfoot twitpic com zl awww bummer shoulda got david carr day d
1             0  ...        upset t update facebook texting cry result school today blah
2             0  ...                    kenichan dived time ball managed save rest bound
3             0  ...                                           body feel itchy like fire
4             0  ...                                      nationwideclass behaving mad t
...         ...  ...                                                                 ...
1599995       4  ...                                     woke having school best feeling
1599996       4  ...            thewdb com cool hear old walt interview bli

In [0]:
%%time
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import nltk
nltk.download('punkt')

# Creating the Bag of Words model 
word2count = {}
for data in df['PreprocessedText'].values: 
    words = nltk.word_tokenize(data) 
    for word in words: 
        if word not in word2count.keys(): 
            word2count[word] = 1
        else: 
            word2count[word] += 1

print(word2count["like"], word2count["hate"])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
29789 8122
CPU times: user 1min 1s, sys: 10.8 ms, total: 1min 1s
Wall time: 1min 1s


In [0]:
%%time
import heapq
freq_words = heapq.nlargest(500, word2count, key=word2count.get)
print(freq_words)

['t', 'day', 'good', 'going', 'love', 'like', 'normal', 'denial', 'got', 'work', 'today', 'time', 'laughing', 'want', 'oh', 'loud', 'know', 'com', 'night', 'Instant', 'Message', 'think', 'new', 'thanks', 'okay', 'home', 'need', 'll', 'yeah', 'tomorrow', 'miss', 'morning', 'feel', 'twitter', 'hope', 'haha', 'great', 'right', 'way', 'wish', 'sleep', 'week', 'fun', 'sad', 'friend', 'come', 'bad', 'sorry', 'tonight', 'thing', 've', 'nice', 'people', 'bit', 'd', 'hour', 'happy', 'getting', 'watching', 'school', 'look', 'thank', 'better', 'yes', 'bed', 'wait', 'hate', 'world', 'didn', 'twitpic', 'year', 'weekend', 'life', 'little', 'picture', 'x', 'guy', 'hey', 'soon', 'web', 'best', 'tweet', 'awesome', 'f', 'cant', 'let', 'long', 'girl', 'working', 'as', 'watch', 'tired', 'lot', 'movie', 'p', 'sick', 'having', 'suck', 'cool', 'feeling', 'god', 'sure', 'game', 'find', 'old', 'birthday', 'man', 'ready', 'won', 'yay', 'looking', 'house', 'phone', 's', 'song', 'went', 'start', 'pretty', 'stop',

In [0]:
%%time
X = [] 
i = 0
for data in df['PreprocessedText'].values: 
    vector = []
    for word in freq_words: 
        if word in nltk.word_tokenize(data): 
            vector.append(1) 
        else: 
            vector.append(0) 
    i+=1
    print(i)
    X.append(vector)


Streaming output truncated to the last 5000 lines.
595004
595005
595006
595007
595008
595009
595010
595011
595012
595013
595014
595015
595016
595017
595018
595019
595020
595021
595022
595023
595024
595025
595026
595027
595028
595029
595030
595031
595032
595033
595034
595035
595036
595037
595038
595039
595040
595041
595042
595043
595044
595045
595046
595047
595048
595049
595050
595051
595052
595053
595054
595055
595056
595057
595058
595059
595060
595061
595062
595063
595064
595065
595066
595067
595068
595069
595070
595071
595072
595073
595074
595075
595076
595077
595078
595079
595080
595081
595082
595083
595084
595085
595086
595087
595088
595089
595090
595091
595092
595093
595094
595095
595096
595097
595098
595099
595100
595101
595102
595103
595104
595105
595106
595107
595108
595109
595110
595111
595112
595113
595114
595115
595116
595117
595118
595119
595120
595121
595122
595123
595124
595125
595126
595127
595128
595129
595130
595131
595132
595133
595134
595135
595136
595137
595138
5951

In [0]:
Xdf = pd.DataFrame(X, columns = freq_words)
print(Xdf)

        t  day  good  going  love  ...  chocolate  listen  gym  saying  vote
0       0    1     0      0     0  ...          0       0    0       0     0
1       1    0     0      0     0  ...          0       0    0       0     0
2       0    0     0      0     0  ...          0       0    0       0     0
3       0    0     0      0     0  ...          0       0    0       0     0
4       1    0     0      0     0  ...          0       0    0       0     0
...    ..  ...   ...    ...   ...  ...        ...     ...  ...     ...   ...
599995  0    0     0      0     0  ...          0       0    0       0     0
599996  0    0     0      0     0  ...          0       0    0       0     0
599997  0    0     0      0     0  ...          0       0    0       0     0
599998  0    0     0      0     0  ...          0       0    0       0     0
599999  0    0     0      0     0  ...          0       0    0       0     0

[600000 rows x 500 columns]


In [0]:
df_traintest = Xdf
print(df_traintest)
y = df['target']
print(y)

        t  day  good  going  love  ...  chocolate  listen  gym  saying  vote
0       0    1     0      0     0  ...          0       0    0       0     0
1       1    0     0      0     0  ...          0       0    0       0     0
2       0    0     0      0     0  ...          0       0    0       0     0
3       0    0     0      0     0  ...          0       0    0       0     0
4       1    0     0      0     0  ...          0       0    0       0     0
...    ..  ...   ...    ...   ...  ...        ...     ...  ...     ...   ...
599995  0    0     0      0     0  ...          0       0    0       0     0
599996  0    0     0      0     0  ...          0       0    0       0     0
599997  0    0     0      0     0  ...          0       0    0       0     0
599998  0    0     0      0     0  ...          0       0    0       0     0
599999  0    0     0      0     0  ...          0       0    0       0     0

[600000 rows x 500 columns]
0          0
1          0
2          0
3       

In [0]:
%%time
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(df_traintest, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(480000, 500) (480000,)
(120000, 500) (120000,)
CPU times: user 2.5 s, sys: 7 ms, total: 2.5 s
Wall time: 2.5 s


In [0]:
%%time
#fit a model
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=100, random_state=0)
random_forest.fit(X_train, y_train.values.ravel())

predictions = random_forest.predict(X_test)
print(predictions)

print("Score:", random_forest.score(X_test, y_test))

[4 4 0 ... 0 0 4]
Score: 0.7096666666666667
CPU times: user 21min 54s, sys: 4.83 s, total: 21min 59s
Wall time: 22min 10s


In [0]:
import pickle
filename = 'random_forest.pkl'
pickle.dump(random_forest, open(filename, 'wb'))

In [0]:
%%time
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train.values.ravel())
print("Score Linear Regression     :", lr.score(X_test, y_test.values.ravel()))

Score Linear Regression     : 0.2533390096100272
CPU times: user 48.5 s, sys: 6.96 s, total: 55.5 s
Wall time: 17.2 s


In [0]:
filename = 'linear_regression.pkl'
pickle.dump(lr, open(filename, 'wb'))

In [0]:
%%time
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_train, y_train.values.ravel())
print("Score DecisionTree            :", dt.score(X_test, y_test.values.ravel()))

Score DecisionTree            : 0.6744583333333334
CPU times: user 5min 8s, sys: 31.5 ms, total: 5min 8s
Wall time: 5min 10s


In [0]:
filename = 'decision_tree.pkl'
pickle.dump(dt, open(filename, 'wb'))

In [0]:
%%time
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=25)
knn.fit(X_train, y_train.values.ravel())
print("Score KNeighbors            :", knn.score(X_test, y_test.values.ravel()))

In [0]:
filename = 'k_neighbours.pkl'
pickle.dump(knn, open(filename, 'wb'))

In [0]:
%%time
from sklearn import svm
supportvm = svm.SVC() 
supportvm.fit(X_train, y_train.values.ravel())
print("Score Support Vector Machine:", supportvm.score(X_test, y_test.values.ravel()))

In [0]:
filename = 'SVM.pkl'
pickle.dump(supportvm, open(filename, 'wb'))

In [0]:
from google.colab import files
#files.download('SVM.pkl')
files.download('decision_tree.pkl')
files.download('k_neighbours.pkl')
files.download('random_forest.pkl')

In [0]:
%%time
df = pd.read_json(r"/content/gdrive/My Drive/PROJECT | S7-S8/Data/AMAZON_FASHION_5.json", lines = True)
df.groupby('asin').mean()
df_test = (df.loc[df['asin'] == 'B0014F7B98'])
print(df_test.head())
print(df_test.columns)
df_test["PreprocessedReviews"] = df_test.reviewText.apply(lambda x: text_cleaner(str(x)))
print(df_test)

X = [] 
for data in df_test['PreprocessedReviews'].values: 
    vector = []
    for word in freq_words: 
        if word in nltk.word_tokenize(data): 
            vector.append(1) 
        else: 
            vector.append(0) 
    X.append(vector)

Xdf = pd.DataFrame(X, columns = freq_words)
print(Xdf)


In [0]:
%%time
print(random_forest.predict(Xdf))
print(dt.predict(Xdf))
print(knn.predict(Xdf))
print(supportvm.predict(Xdf))